<a href=https://experimental-design.github.io/bofire/>
  <img width="350" src="https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/logos/bofire-long.png" alt="BoFire Logo" />
</a>

## Getting started with BoFire

In our [docs](https://experimental-design.github.io/bofire/), you can find all different options for the [BoFire installation](https://experimental-design.github.io/bofire/install/). For basic BoFire Bayesian optimization features using [BoTorch](https://botorch.org/) which depends on
[PyTorch](https://pytorch.org/), you need to run

```
pip install bofire[optimization]
```

For a more complete introduction to BoFire, please look in our [docs](https://experimental-design.github.io/bofire/).

### Optimization problem

Let us consider a test function for single-objective optimization - the [Himmelblau's function](https://en.wikipedia.org/wiki/Himmelblau%27s_function). The Himmelblau's function has four identical local minima used to test the performance of optimization algorithms. The optimization domain of the Himmelblau's function is illustrated below together with the four minima marked red.

[comment]: <> (TODO: Exchange link below with https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/tutorials/himmelblau.png)

<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/tutorials/himmelblau.png"
    alt="Himmelblau's function"
    width="300"/>
</div>


### Defining the optimization output

Let's consider the single continuous output variable *y* of the Himmelblau's function with the objective to minimize it. In BoFire's terminology, we create a `MinimizeObjective` object to define the optimization objective of a `Continuous Output` feature.

In [1]:
from bofire.data_models.features.api import ContinuousOutput
from bofire.data_models.objectives.api import MinimizeObjective


objective = MinimizeObjective()
output_feature = ContinuousOutput(key="y", objective=objective)

For more details on `Output` features and `Objective` objects, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining the optimization inputs

For the two continuous input variables of the Himmelblau's function *x1* and *x2*, we create two `ContinuousInput` features including boundaries following BoFire's terminology.

In [2]:
from bofire.data_models.features.api import ContinuousInput


input_feature_1 = ContinuousInput(key="x1", bounds=(-5, 5))
input_feature_2 = ContinuousInput(key="x2", bounds=(-5, 5))

For more details on `Input` features, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining the optimization domain

In BoFire's terminology, `Domain` objects fully describe the search space of the optimization problem. `Input` and `Output` features are optionally bound with `Constraint` objects to specify allowed relationships between the parameters. Here, we will run an unconstrained optimization. For more details, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


In [3]:
from bofire.data_models.domain.api import Domain, Inputs, Outputs


domain = Domain(
    inputs=Inputs(features=[input_feature_1, input_feature_2]),
    outputs=Outputs(features=[output_feature]),
)

### Draw candidates and execute experiments

Let's define the Himmelblau's function to evaluate points in the domain space.

In [4]:
def himmelblau(x1, x2):
    return (x1**2 + x2 - 11) ** 2 + (x1 + x2**2 - 7) ** 2


To initialize an iterative Bayesian optimization loop, let's first randomly draw 10 samples from the domain. In BoFire's terminology, those suggested samples are called `Candidates`.

In [5]:
candidates = domain.inputs.sample(10, seed=13)

print(candidates)

         x1        x2
0  1.059211  1.374497
1 -4.176967 -1.589341
2 -3.784766  4.674178
3 -2.449681  1.128971
4  3.055689 -2.594837
5 -4.779897  3.157958
6 -0.040224 -2.291381
7  3.854749  3.860476
8 -0.896600  1.240756
9  0.536098 -0.041359


Let's evaluate the function output for the randomly drawn candidates using the `himmelblau` function to obtain `Experiments` in BoFire's terminology.

In [6]:
experimental_output = candidates.apply(
    lambda row: himmelblau(row["x1"], row["x2"]), axis=1
)

experiments = candidates.copy()
experiments["y"] = experimental_output

print(experiments)

         x1        x2           y
0  1.059211  1.374497   88.725854
1 -4.176967 -1.589341   98.436483
2 -3.784766  4.674178  186.371901
3 -2.449681  1.128971   81.809936
4  3.055689 -2.594837   25.904959
5 -4.779897  3.157958  228.427101
6 -0.040224 -2.291381  179.821177
7  3.854749  3.860476  197.842843
8 -0.896600  1.240756  120.611344
9  0.536098 -0.041359  157.407519


For more details on candidates and experiments, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Defining an optimization strategy

Let's specify the strategy how the Bayesian optimization campaign should be conducted. Here, we define a single-objective Bayesian optimization strategy and pass the optimization domain together with a acquisition function. Here, we use logarithmic expected improvement `qLogEI` as the acquisition function. In BoFire's terminology, we create a serializable data model `SoboStrategy` which we then map to our functional model.


In [7]:
import bofire.strategies.api as strategies
from bofire.data_models.acquisition_functions.api import qLogEI
from bofire.data_models.strategies.api import SoboStrategy


sobo_strategy_data_model = SoboStrategy(
    domain=domain, acquisition_function=qLogEI(), seed=19
)

sobo_strategy = strategies.map(sobo_strategy_data_model)

c:\Users\chrihaas\AppData\Local\miniforge3\envs\test12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


For more details on strategy data models and functional models, see the respective sections in our [docs](https://experimental-design.github.io/bofire/).


### Run the optimization loop

To run the optimization loop using BoFire's terminology, we first `tell` the strategy object about the experiments we have already executed.


In [8]:
sobo_strategy.tell(experiments=experiments)

We run the optimization loop for 30 iterations. In each iteration, we `ask` the strategy object to suggest one new candidate, which is returned as a list containing a single item. We then perform a new experiment by evaluating the Himmelblau function output of this candidate. After completing the experiment, we add the new data to our existing experiments and `tell` the strategy object about the updated dataset. This process is repeated for each of the 30 iterations.

In [9]:
import pandas as pd


for _ in range(30):
    new_candidates = sobo_strategy.ask(candidate_count=1)

    new_experiments = new_candidates.copy()
    new_experiments["y"] = new_candidates.apply(
        lambda row: himmelblau(row["x1"], row["x2"]), axis=1
    )

    sobo_strategy.tell(experiments=new_experiments)

The optimization behavior of the strategy is shown in the animated figure below. The four minima are marked red, the experiments carried out are marked blue with blue lines connecting them. The contours are indicating the predicted mean of the current model of each iteration.

[comment]: <> (TODO: Update Figure with Dorofee and exchange link below with https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/tutorials/himmelblau_optimization.gif)

<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/experimental-design/bofire/main/graphics/tutorials/himmelblau_optimization.gif"
    alt="Optimization of Himmelblau's function"
    width="300"/>
</div>